# Wind Downscaling

## Prerequisites

* Conda environment
* Get a Copernicus API key from: https://cds.climate.copernicus.eu/api-how-to
  * create a file at \$HOME/.cdsapirc with the required UID and key

## Install the required packages

In [1]:
import os
from datetime import date, datetime
from pathlib import Path

In [3]:
!conda install -y -c conda-forge gdal tensorflow xarray numpy=1.19.5 pandas pysftp cdsapi elevation rasterio dask python-dotenv

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.10.3
  latest version: 4.11.0

Please update conda by running

    $ conda update -n base conda



# All requested packages already installed.



In [4]:
!pip install topo-descriptors

In [2]:
!pip uninstall downscaling -y
!pip install -U git+https://github.com/OpheliaMiralles/wind-downscaling-gan.git

Found existing installation: downscaling 1.0
Uninstalling downscaling-1.0:
  Successfully uninstalled downscaling-1.0
  Cloning https://github.com/OpheliaMiralles/wind-downscaling-gan.git to /private/var/folders/nb/drg15dgx4mj6bqb_f4c9bnvc0000gp/T/pip-req-build-qs5q7054
  Running command git clone --filter=blob:none -q https://github.com/OpheliaMiralles/wind-downscaling-gan.git /private/var/folders/nb/drg15dgx4mj6bqb_f4c9bnvc0000gp/T/pip-req-build-qs5q7054
  Resolved https://github.com/OpheliaMiralles/wind-downscaling-gan.git to commit fa87f976cd1dee51233d5ef770b142f87bb7096c
  Preparing metadata (setup.py) ... done
  Created wheel for downscaling: filename=downscaling-1.0-py3-none-any.whl size=34921926 sha256=c31b4b46197938fe63b99e8659f3ef89acff35711c45d31cd1cdc1bda5f6a1a0
  Stored in directory: /private/var/folders/nb/drg15dgx4mj6bqb_f4c9bnvc0000gp/T/pip-ephem-wheel-cache-q5pqwfmd/wheels/5f/c9/1a/f3d704b062d6cea2f7f70735392d0bf114ed90dcad1d16d103
Successfully built downscaling


## Set configuration

In [5]:
DATA_ROOT = Path('./data') and Path('/Volumes/ExtremeSSD/data')
ERA5_DATA_FOLDER = DATA_ROOT / 'ERA5_FR'
DEM_DATA_FILE = DATA_ROOT / 'dem/France-90m-DEM.tif'
START_DATE = date(2016,4,1)
END_DATE = date(2016,4,2)
LONGITUDE_R = (-4.96, 8.3)
LATITUDE_R = (42.2, 51.3)

## Data Loading

In [6]:
from downscaling.data import download_ERA5
download_ERA5(ERA5_DATA_FOLDER, START_DATE, END_DATE, LATITUDE_R, LONGITUDE_R)
print('Done')

File 20160401_era5_surface_hourly already exists
File 20160402_era5_surface_hourly already exists
File 20160401_era5_z500_hourly already exists
File 20160402_era5_z500_hourly already exists
Done


In [7]:
if not DEM_DATA_FILE.exists():
    dest = str(DEM_DATA_FILE)
    !eio --product SRTM3 clip -o {dest} --bounds -4.96 42.2 8.3 51.3

## Downscaling

In [8]:
from downscaling import downscale, plot_wind_fields
import xarray as xr

In [ ]:
era5 = xr.open_mfdataset(ERA5_DATA_FOLDER.glob('20160401*surface*.nc'))
raster_topo = xr.open_rasterio(DEM_DATA_FILE)
downscaled_maps = downscale(era5, raster_topo, range_lon=LONGITUDE_R, range_lat=LATITUDE_R)

Loading network...
Applying model to 1935 patches
Predicted 1%
Predicted 2%
Predicted 2%
Predicted 3%
Predicted 4%
Predicted 5%
Predicted 6%
Predicted 7%
Predicted 7%
Predicted 8%
Predicted 9%
Predicted 10%
Predicted 11%
Predicted 12%
Predicted 12%
Predicted 13%
Predicted 14%
Predicted 15%
Predicted 16%
Predicted 17%
Predicted 17%
Predicted 18%
Predicted 19%
Predicted 20%
Predicted 21%
Predicted 21%
Predicted 22%
Predicted 23%
Predicted 24%
Predicted 25%
Predicted 26%
Predicted 26%
Predicted 27%
Predicted 28%
Predicted 29%
Predicted 30%
Predicted 31%
Predicted 31%
Predicted 32%
Predicted 33%
Predicted 34%
Predicted 35%
Predicted 36%
Predicted 36%


In [ ]:
fig = plot_wind_fields(downscaled_maps.isel(time=0), range_lon=LONGITUDE_R, range_lat=LATITUDE_R)
fig.show()

## Downscaling wind fields in Switzerland with target COSMO-1

In [ ]:
from downscaling import process_era5, process_topo, predict

In [ ]:
ERA5_DATA_FOLDER = DATA_ROOT / 'ERA5'
DEM_DATA_FILE = DATA_ROOT / 'dem/Switzerland-90m-DEM.tif'
START_DATE = date(2016,4,1)
END_DATE = date(2016,4,1)
LONGITUDE_R = (5.8, 10.6)
LATITUDE_R = (45.75, 47.9)

In [ ]:
import requests
r = requests.get('https://raw.githubusercontent.com/OpheliaMiralles/wind-downscaling-gan/master/src/downscaling/switzerland_cosmo_map.nc')
with open('switzerland_cosmo_map.nc', 'wb') as f:
    f.write(r.content)
HIGH_RES_TEMPLATE = xr.open_dataset('switzerland_cosmo_map.nc')

In [ ]:
era5 = process_era5(xr.open_dataset(ERA5_DATA_FOLDER / '20171005_era5_surface_hourly.nc'), high_res_template=HIGH_RES_TEMPLATE)
topo = process_topo(xr.open_rasterio(DEM_DATA_FILE), high_res_template=HIGH_RES_TEMPLATE)

In [ ]:
downscaled_maps = predict(era5, topo, high_res_template=HIGH_RES_TEMPLATE)

## Plots of wind fields in Switzerland

In [ ]:
from downscaling import plot_elevation, plot_wind_fields
from cartopy.crs import epsg
HIGH_RES_CRS = epsg(21781)  # crs for the target map, for plots

In [ ]:
raw_topo = xr.open_rasterio(DEM_DATA_FILE)
fig_topo = plot_elevation(raw_topo, range_lon=LONGITUDE_R, range_lat=LATITUDE_R)
fig_topo.show()

In [ ]:
fig = plot_wind_fields(downscaled_maps.isel(time=0), range_lon=LONGITUDE_R, range_lat=LATITUDE_R, high_res_crs=HIGH_RES_CRS)
fig.show()